# Fill Fitting 

This section performs mass fitting on individual Fill files for both **B2OC** (B⁺ → D⁰π⁺) and **B2CC** (B⁺ → J/ψK⁺) decay modes.

**Fitting Method**: Double-sided Crystal Ball PDFs for signal + exponential background

**Output**: 
- New `fit_results` TTree containing all fit parameters and uncertainties
- Canvas objects with mass plots showing data and fitted components

The workflow consists of three cells: cleanup, fitting loop, and visualization.

In [9]:
import ROOT as r
import numpy as np
import matplotlib.pyplot as plt
import math
from ROOT import RooFit
from ROOT import TChain
from ROOT import RooStats
from datetime import datetime
from uncertainties import ufloat
from scipy.optimize import curve_fit
from pathlib import Path

# Define data directories for organized file management
DATA_CLEAN = Path("data/processed_clean_bp_p")   

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
print(r.gROOT.GetVersion())

%jsroot on

c = r.TCanvas()

6.34.04


In [29]:
from array import array
import re                               # regular expressions for filename parsing
r.EnableImplicitMT()                    # enable multi-threading in RooFit

# Input files to process (fill-level B⁺ → D⁰π⁺ data)
files = [
    DATA_CLEAN/"2024_B2OC_B8_F10215.root",
]

# Observable: B+ mass variable  
x = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)    # MeV/c² range

# B+ → D⁰π+ signal parameters (main peak around 5278 MeV)
mean = r.RooRealVar("mean", "mean", 5278.46, 5250, 5300)

# Crystal Ball 1 parameters
alpha = r.RooRealVar("alpha", "alpha", 1.25015, 1, 2)
n = r.RooRealVar("n", "n", 2.40748, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 11, 1.0, 30)  
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

# Background model
tau = r.RooRealVar("tau", "Decay constant", -0.00219122, -1, 0)  # exponential slope
background = r.RooExponential("background", "Exponential background", x, tau)

# Crystal Ball 2 parameters
alpha_2 = r.RooRealVar("alpha2", "alpha2", -2.2701, -20, -0.01)
n_2 = r.RooRealVar("n2", "n2", 2.47972, 0.05, 50)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 12.7184, 1.0, 20)   
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.65, 0.0, 1.0)

# Parameter constraints: fix shape parameters, float yields and resolutions
mean.setConstant(False)          # allow mean to float
alpha.setConstant(True)          # fix tail parameter
n.setConstant(True)              # fix tail parameter
cb_sigma.setConstant(True)       # fix resolution
tau.setConstant(False)           # allow background slope to float
alpha_2.setConstant(True)        # fix tail parameter
n_2.setConstant(True)            # fix tail parameter
cb_sigma_2.setConstant(False)    # allow resolution to float
frac_cb_2.setConstant(False)     # allow CB fraction to float

# Helper functions for branch management
def make_branch(tree, name, var_type='d'):
    """Helper function to create a branch with array buffer"""
    if var_type == 'd':
        buf = array('d', [0.])
        tree.Branch(name, buf, f"{name}/D")
    elif var_type == 'i':
        buf = array('i', [0])
        tree.Branch(name, buf, f"{name}/I")
    return buf

def setaddr(tree, name, buf): 
    """Helper function to set branch addresses safely"""
    branch = tree.GetBranch(name)
    if branch: 
        branch.SetAddress(buf)

# Main fitting loop over input files
for fname in files:

    print(f"\n▶ fitting fill file {fname}")

    # Open file and get tree
    f = r.TFile.Open(str(fname), "UPDATE")          # UPDATE keeps existing content
    tree = f.Get("ST-b2oc")
    if not tree:
        print("ST-b2oc tree not found, skipping");  f.Close();  continue

    # Load data into RooDataSet
    data = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt = data.numEntries()
    print(f"entries in tree = {Nevt}")

    # Define yield variables with initial estimates
    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)   # ~25% signal
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)   # ~75% background
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    
    # Construct composite PDF
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    # Total model: signal + background
    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    # Perform fit
    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    # Print fit results
    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    
    # Error propagation with uncertainties package
    sig_u = ufloat(nsig.getVal(), nsig.getError()) # background fraction, propagated uncertainty 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # Store results in TTree branches
    f.cd()
    res_tree = f.Get("fit_results")
    
    # Create results tree if it doesn't exist
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")
        
        # Create branches using helper function
        br_mean           = make_branch(res_tree, "mean")
        br_mean_e         = make_branch(res_tree, "mean_err")
        br_alpha          = make_branch(res_tree, "alpha1")
        br_alpha_e        = make_branch(res_tree, "alpha1_err")
        br_n              = make_branch(res_tree, "n1")
        br_n_e            = make_branch(res_tree, "n1_err")
        br_cb_sigma       = make_branch(res_tree, "cb_sigma1")
        br_cb_sigma_e     = make_branch(res_tree, "cb_sigma1_err")
        br_tau            = make_branch(res_tree, "tau")
        br_tau_e          = make_branch(res_tree, "tau_err")
        br_alpha_2        = make_branch(res_tree, "alpha2")
        br_alpha_2_e      = make_branch(res_tree, "alpha2_err")
        br_n_2            = make_branch(res_tree, "n2")
        br_n_2_e          = make_branch(res_tree, "n2_err")
        br_cb_sigma_2     = make_branch(res_tree, "cb_sigma2")
        br_cb_sigma_2_e   = make_branch(res_tree, "cb_sigma2_err")
        br_frac_cb_2      = make_branch(res_tree, "frac_cb_2")
        br_frac_cb_2_e    = make_branch(res_tree, "frac_cb_2_err")
        br_Nevt           = make_branch(res_tree, "Nevt")          
        br_nsig           = make_branch(res_tree, "nsig")
        br_nsig_e         = make_branch(res_tree, "nsig_err")
        br_nbkg           = make_branch(res_tree, "nbkg")
        br_nbkg_e         = make_branch(res_tree, "nbkg_err")
        br_ntot           = make_branch(res_tree, "ntot")
        br_ntot_e         = make_branch(res_tree, "ntot_err")
        
        # Integer branches for fit quality
        br_status         = make_branch(res_tree, "status", 'i')
        br_covqual        = make_branch(res_tree, "covqual", 'i')
        
    else:
        # Tree exists: create arrays and set branch addresses using helper functions
        br_mean           = array('d', [0.]); br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.]); br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.]); br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.]); br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.]); br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.]); br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.]); br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.]); br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.]); br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.]); br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.]); br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.]); br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0]); br_covqual        = array('i', [0])

        # Set all branch addresses using dictionary approach with helper function
        branch_mapping = {
            "mean": br_mean, "mean_err": br_mean_e,
            "alpha1": br_alpha, "alpha1_err": br_alpha_e,
            "n1": br_n, "n1_err": br_n_e,
            "cb_sigma1": br_cb_sigma, "cb_sigma1_err": br_cb_sigma_e,
            "tau": br_tau, "tau_err": br_tau_e,
            "alpha2": br_alpha_2, "alpha2_err": br_alpha_2_e,
            "n2": br_n_2, "n2_err": br_n_2_e,
            "cb_sigma2": br_cb_sigma_2, "cb_sigma2_err": br_cb_sigma_2_e,
            "frac_cb_2": br_frac_cb_2, "frac_cb_2_err": br_frac_cb_2_e,
            "Nevt": br_Nevt,
            "nsig": br_nsig, "nsig_err": br_nsig_e,
            "nbkg": br_nbkg, "nbkg_err": br_nbkg_e,
            "ntot": br_ntot, "ntot_err": br_ntot_e,
            "status": br_status, "covqual": br_covqual
        }
        
        for branch_name, buffer in branch_mapping.items():
            setaddr(res_tree, branch_name, buffer)

    # Fill branch values with fit results
    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal()
    br_nsig_e[0]          = nsig.getError()
    br_nbkg[0]            = nbkg.getVal()
    br_nbkg_e[0]          = nbkg.getError()
    
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n
    br_ntot_e[0]          = ntot_u.s
    br_status[0]          = fit_res.status()
    br_covqual[0]         = fit_res.covQual()

    # Store fit results in branch arrays
    res_tree.Fill()

    # Create and save fit plots
    canvas = r.TCanvas(f"canvas_{fname.stem}", "Dpi mass fit", 1000, 750)
    frame = x.frame(r.RooFit.Title(f"Dpi mass fit: {fname.stem}"))

    # Plot data and model components
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))

    # Format and draw the plot
    title_latex = "B^{+} #rightarrow #bar{D}^{0}#pi^{+}"
    frame.SetTitle(f"{title_latex} Mass Fit: {fname.stem}")
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    
    # Add legend for plot components
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total fit", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    legend.AddEntry(0, "Background", "l")  
    legend.Draw()

    # Calculate automatic y-axis minimum for log scale
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Apply log scale and save PNG
    frame.SetMinimum(ymin * 0.8)
    canvas.SetLogy()
    canvas.Write(f"canvas_{fname.stem}_log")
    
    # Save log-scale PNG to output directory
    canvas.SetCanvasSize(2000, 1500)
    log_plot_path = DATA_CLEAN / f"dpi_fit_{fname.stem}_log.png"
    canvas.SaveAs(str(log_plot_path))
    print(f"saved log-scale plot: {log_plot_path}")

    # Save results and clean up
    res_tree.Write("", r.TObject.kOverwrite)
    f.Close()
    
    print(f"Completed processing {fname.name}\n")




▶ fitting fill file data/processed_clean_bp_p/2024_B2OC_B8_F10215.root
entries in tree = 7623

=== fitted yields ===
nsig  = 1857 ± 97
nbkg  = 5766 ± 115
ntot  = 7623 (derived)
0.9803701389199735
background fraction = 0.756371+/-0.010314
fit status = 0, covQual = 3
saved log-scale plot: data/processed_clean_bp_p/dpi_fit_2024_B2OC_B8_F10215_log.png
Completed processing 2024_B2OC_B8_F10215.root

[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #0 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5645.79
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #1 because Bp_DTF_OwnPV_MASS cannot accommodate the value 4974.16
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #2 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5002.08
[#1] INFO:DataHandling -- Ro

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -19061.86849 Edm =       56.40255136 NCalls =     25
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 0.455593
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -19061.86849
  Edm           : 5205.344801
  Internal parameters:	[     0.2356918764      0.304692654     0.1388456842    -0.1433475689     0.1433475689      1.477141165]	
  Internal gradient  :	[     -116.9198525      -10.7654967     -81.27174526     -35.47877952      106.4362692      -1397.74526]	
  Internal covariance matrix:
[[      1.1649587      6.6676736    -0.12665301    -0.22844884     0.28496567  -0.0027564801]
 [      6.6676736      38.324874    -0.72793103     -1.3062642      1.6294213   -0.016017817]
 [    -0.12665301    -0.72793103    0.014546972    0.024800494   -0.030935903   0.0002957012]
 [    -0.228448

In [30]:
from array import array
import re                               # regular expressions for filename parsing
r.EnableImplicitMT()                    # enable multi-threading in RooFit

# Input files to process (fill-level B⁺ → J/ψK⁺ data)
files = [
DATA_CLEAN/"2024_B2CC_B8_F10215.root",
]

# Observable: B+ mass variable MeV/c² range
x = r.RooRealVar("Bp_DTF_OwnPV_MASS", "B^{+} mass", 5200, 5450)   

# B+ → J/ψK+ signal parameters (main peak around 5279 MeV)
mean = r.RooRealVar("mean", "mean", 5279.38, 5250, 5300)

# Background model
tau = r.RooRealVar("tau", "Decay constant", -0.00107319, -1, 0) 
background = r.RooExponential("background", "Exponential background", x, tau)

# Crystal Ball 1 parameters
alpha = r.RooRealVar("alpha", "alpha", 1.59501, 0.1, 2)
n = r.RooRealVar("n", "n", 2.75673, 0.5, 5)
cb_sigma = r.RooRealVar("cb_sigma", "cb_sigma", 7.2753, 5, 10)
crystal_ball = r.RooCBShape("crystal_ball", "Crystal ball PDF", x, mean, cb_sigma, alpha, n)

# Crystal Ball 2 parameters
alpha_2 = r.RooRealVar("alpha2", "alpha2", -1.29179, -1.5, -0.1)
n_2 = r.RooRealVar("n2", "n2", 5.29, 0.5, 10)
cb_sigma_2 = r.RooRealVar("cb_sigma2", "cb_sigma2", 8.21279, 5, 10)
crystal_ball_2 = r.RooCBShape("crystal_ball2", "Crystal ball PDF 2", x, mean, cb_sigma_2, alpha_2, n_2)

frac_cb_2 = r.RooRealVar("frac_cb_2", "Fraction of crystal ball 2", 0.5, 0.0, 1.0)

# Parameter constraints: fix shape parameters, float yields and resolutions
mean.setConstant(False)          # allow mean to float
alpha.setConstant(True)          # fix tail parameter
n.setConstant(True)              # fix tail parameter
cb_sigma.setConstant(True)       # fix resolution
tau.setConstant(False)           # allow background slope to float
alpha_2.setConstant(True)        # fix tail parameter
n_2.setConstant(True)            # fix tail parameter
cb_sigma_2.setConstant(False)    # allow resolution to float
frac_cb_2.setConstant(False)     # allow CB fraction to float

# Helper functions for branch management
def make_branch(tree, name, var_type='d'):
    """Helper function to create a branch with array buffer"""
    if var_type == 'd':
        buf = array('d', [0.])
        tree.Branch(name, buf, f"{name}/D")
    elif var_type == 'i':
        buf = array('i', [0])
        tree.Branch(name, buf, f"{name}/I")
    return buf

def setaddr(tree, name, buf): 
    """Helper function to set branch addresses safely"""
    branch = tree.GetBranch(name)
    if branch: 
        branch.SetAddress(buf)

# Main fitting loop over input files
for fname in files:

    print(f"\n▶ fitting fill file {fname}")

    # Open file and get tree, UPDATE keeps existing content
    f = r.TFile.Open(str(fname), "UPDATE")        
    tree = f.Get("ST-b2cc")
    if not tree:
        print("ST-b2cc tree not found, skipping");  f.Close();  continue

    # Load data into RooDataSet
    data = r.RooDataSet("data", "data", r.RooArgSet(x), r.RooFit.Import(tree))
    Nevt = data.numEntries()
    print(f"entries in tree = {Nevt}")

    # Define yield variables with initial estimates
    nsig = r.RooRealVar("nsig", "yield sig", Nevt * 0.25, Nevt * 0.05, Nevt * 0.4)   # ~25% signal
    nbkg = r.RooRealVar("nbkg", "yield bkg", Nevt * 0.75, Nevt * 0.6, Nevt * 0.95)   # ~75% background
    ntot = r.RooFormulaVar("ntot","@0+@1", r.RooArgList(nsig, nbkg))
    
    # Construct composite PDF
    sig_pdf = r.RooAddPdf("sig","", r.RooArgList(crystal_ball_2, crystal_ball,),
                                    r.RooArgList(frac_cb_2))

    # Total model: signal + background
    model = r.RooAddPdf("model","sig+bkg", r.RooArgList(sig_pdf, background),
                                         r.RooArgList(nsig,    nbkg))

    # Perform fit
    fit_res = model.fitTo(data, r.RooFit.Save(True), r.RooFit.Strategy(2))

    # Print fit results
    print("\n=== fitted yields ===")
    print(f"nsig  = {nsig.getVal():.0f} ± {nsig.getError():.0f}")
    print(f"nbkg  = {nbkg.getVal():.0f} ± {nbkg.getError():.0f}")
    print(f"ntot  = {ntot.getVal():.0f} (derived)")
    print(frac_cb_2.getVal())
    
    # Error propagation with uncertainties package
    sig_u = ufloat(nsig.getVal(), nsig.getError()) 
    bkg_u = ufloat(nbkg.getVal(), nbkg.getError())
    frac_bkg = bkg_u / (sig_u + bkg_u)

    print(f"background fraction = {frac_bkg:.6f}")
    print(f"fit status = {fit_res.status()}, covQual = {fit_res.covQual()}")

    # Store results in TTree branches
    f.cd()
    res_tree = f.Get("fit_results")
    
    # Create results tree if it doesn't exist
    if not res_tree:
        res_tree = r.TTree("fit_results", "Mass-fit results per fill")
        
        # Create branches using helper function
        br_mean           = make_branch(res_tree, "mean")
        br_mean_e         = make_branch(res_tree, "mean_err")
        br_alpha          = make_branch(res_tree, "alpha1")
        br_alpha_e        = make_branch(res_tree, "alpha1_err")
        br_n              = make_branch(res_tree, "n1")
        br_n_e            = make_branch(res_tree, "n1_err")
        br_cb_sigma       = make_branch(res_tree, "cb_sigma1")
        br_cb_sigma_e     = make_branch(res_tree, "cb_sigma1_err")
        br_tau            = make_branch(res_tree, "tau")
        br_tau_e          = make_branch(res_tree, "tau_err")
        br_alpha_2        = make_branch(res_tree, "alpha2")
        br_alpha_2_e      = make_branch(res_tree, "alpha2_err")
        br_n_2            = make_branch(res_tree, "n2")
        br_n_2_e          = make_branch(res_tree, "n2_err")
        br_cb_sigma_2     = make_branch(res_tree, "cb_sigma2")
        br_cb_sigma_2_e   = make_branch(res_tree, "cb_sigma2_err")
        br_frac_cb_2      = make_branch(res_tree, "frac_cb_2")
        br_frac_cb_2_e    = make_branch(res_tree, "frac_cb_2_err")
        br_Nevt           = make_branch(res_tree, "Nevt")          
        br_nsig           = make_branch(res_tree, "nsig")
        br_nsig_e         = make_branch(res_tree, "nsig_err")
        br_nbkg           = make_branch(res_tree, "nbkg")
        br_nbkg_e         = make_branch(res_tree, "nbkg_err")
        br_ntot           = make_branch(res_tree, "ntot")
        br_ntot_e         = make_branch(res_tree, "ntot_err")
        
        # Integer branches for fit quality
        br_status         = make_branch(res_tree, "status", 'i')
        br_covqual        = make_branch(res_tree, "covqual", 'i')
        
    else:
        # Tree exists: create arrays and set branch addresses using helper functions
        br_mean           = array('d', [0.]); br_mean_e         = array('d', [0.])
        br_alpha          = array('d', [0.]); br_alpha_e        = array('d', [0.])
        br_n              = array('d', [0.]); br_n_e            = array('d', [0.])
        br_cb_sigma       = array('d', [0.]); br_cb_sigma_e     = array('d', [0.])
        br_tau            = array('d', [0.]); br_tau_e          = array('d', [0.])
        br_alpha_2        = array('d', [0.]); br_alpha_2_e      = array('d', [0.])
        br_n_2            = array('d', [0.]); br_n_2_e          = array('d', [0.])
        br_cb_sigma_2     = array('d', [0.]); br_cb_sigma_2_e   = array('d', [0.])
        br_frac_cb_2      = array('d', [0.]); br_frac_cb_2_e    = array('d', [0.])
        br_Nevt           = array('d', [0.])
        br_nsig           = array('d', [0.]); br_nsig_e         = array('d', [0.])
        br_nbkg           = array('d', [0.]); br_nbkg_e         = array('d', [0.])
        br_ntot           = array('d', [0.]); br_ntot_e         = array('d', [0.])
        br_status         = array('i', [0]); br_covqual        = array('i', [0])

        # Set all branch addresses using dictionary approach with helper function
        branch_mapping = {
            "mean": br_mean, "mean_err": br_mean_e,
            "alpha1": br_alpha, "alpha1_err": br_alpha_e,
            "n1": br_n, "n1_err": br_n_e,
            "cb_sigma1": br_cb_sigma, "cb_sigma1_err": br_cb_sigma_e,
            "tau": br_tau, "tau_err": br_tau_e,
            "alpha2": br_alpha_2, "alpha2_err": br_alpha_2_e,
            "n2": br_n_2, "n2_err": br_n_2_e,
            "cb_sigma2": br_cb_sigma_2, "cb_sigma2_err": br_cb_sigma_2_e,
            "frac_cb_2": br_frac_cb_2, "frac_cb_2_err": br_frac_cb_2_e,
            "Nevt": br_Nevt,
            "nsig": br_nsig, "nsig_err": br_nsig_e,
            "nbkg": br_nbkg, "nbkg_err": br_nbkg_e,
            "ntot": br_ntot, "ntot_err": br_ntot_e,
            "status": br_status, "covqual": br_covqual
        }
        
        for branch_name, buffer in branch_mapping.items():
            setaddr(res_tree, branch_name, buffer)

    # 5) fill the branch values

    br_mean[0]            = mean.getVal()
    br_mean_e[0]          = mean.getError()
    br_alpha[0]           = alpha.getVal()
    br_alpha_e[0]         = alpha.getError()
    br_n[0]               = n.getVal()
    br_n_e[0]             = n.getError()
    br_cb_sigma[0]        = cb_sigma.getVal()
    br_cb_sigma_e[0]      = cb_sigma.getError()
    br_tau[0]             = tau.getVal()
    br_tau_e[0]           = tau.getError()
    br_alpha_2[0]         = alpha_2.getVal()
    br_alpha_2_e[0]       = alpha_2.getError()
    br_n_2[0]             = n_2.getVal()
    br_n_2_e[0]           = n_2.getError()
    br_cb_sigma_2[0]      = cb_sigma_2.getVal()
    br_cb_sigma_2_e[0]    = cb_sigma_2.getError()
    br_frac_cb_2[0]       = frac_cb_2.getVal()
    br_frac_cb_2_e[0]     = frac_cb_2.getError()
    br_Nevt[0]            = Nevt
    br_nsig[0]            = nsig.getVal();        br_nsig_e[0]   = nsig.getError()
    br_nbkg[0]            = nbkg.getVal();        br_nbkg_e[0]   = nbkg.getError()
    ntot_u                = sig_u + bkg_u         
    br_ntot[0]            = ntot_u.n;             br_ntot_e[0]   = ntot_u.s
    br_status[0]          = fit_res.status();     br_covqual[0]  = fit_res.covQual()

    res_tree.Fill()

    # 5) make & save the canvas (linear + log)
    canvas = r.TCanvas(f"canvas_{fname.stem}", "JpsiK mass fit", 1000, 750)
    frame = x.frame(r.RooFit.Title(f"JpsiK mass fit: {fname.stem}"))
    
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kWhite), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.Components(background), r.RooFit.FillColor(r.kGreen + 2), r.RooFit.FillStyle(3001), r.RooFit.DrawOption("F"), r.RooFit.LineColor(r.kGreen + 2), r.RooFit.LineStyle(r.kDashed))
    data.plotOn(frame, r.RooFit.MarkerStyle(20), r.RooFit.LineColor(r.kBlack), r.RooFit.DrawOption("PE0"))
    model.plotOn(frame, r.RooFit.LineColor(r.kRed), r.RooFit.Name("total_curve"))
    model.plotOn(frame, r.RooFit.Components(crystal_ball), r.RooFit.LineStyle(2), r.RooFit.LineColor(r.kBlue))
    model.plotOn(frame, r.RooFit.Components(crystal_ball_2), r.RooFit.LineStyle(3), r.RooFit.LineColor(r.kBlue))

    
    # Format and draw the plot
    title_latex = "B^{+} #rightarrow J/#psi K^{+}"
    frame.SetTitle(f"{title_latex} Mass Fit: {fname.stem}")
    frame.GetXaxis().SetTitle("M(B^{+})  [MeV/c^{2}]")
    frame.GetXaxis().CenterTitle(True)
    frame.GetYaxis().CenterTitle(True)
    frame.Draw()
    
    # Add legend for plot components
    legend = r.TLegend(0.72, 0.60, 0.98, 0.88)
    legend.SetTextSize(0.025)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.AddEntry(frame.findObject("total_curve"), "Total fit", "l")
    legend.AddEntry(frame.findObject("h_data"), "Data", "lep")
    legend.AddEntry(0, "Background", "l")  
    legend.Draw()

    # Calculate automatic y-axis minimum for log scale
    x_min_bg = 5400 
    x_max_bg = 5450
    n_bins = 100
    bin_width = (x.getMax() - x.getMin()) / n_bins
    n_bg = data.reduce(f"{x.GetName()} >= {x_min_bg} && {x.GetName()} < {x_max_bg}").numEntries()
    ymin = max(1, n_bg / ((x_max_bg - x_min_bg) / bin_width))

    # Apply log scale and save PNG
    frame.SetMinimum(ymin * 0.8)
    canvas.SetLogy()
    canvas.Write(f"canvas_{fname.stem}_log")

    # Save log-scale PNG to output directory
    canvas.SetCanvasSize(2000, 1500)
    log_plot_path = DATA_CLEAN / f"jpsik_fit_{fname.stem}_log.png"
    canvas.SaveAs(str(log_plot_path))
    print(f"saved log-scale plot: {log_plot_path}")

    # Save results and clean up
    res_tree.Write("", r.TObject.kOverwrite)
    f.Close()

    print(f"Completed processing {fname.name}\n")



▶ fitting fill file data/processed_clean_bp_p/2024_B2CC_B8_F10215.root
entries in tree = 46592

=== fitted yields ===
nsig  = 2654 ± 128
nbkg  = 43943 ± 241
ntot  = 46597 (derived)
0.8249292902041203
background fraction = 0.943052+/-0.002599
fit status = 0, covQual = 3
saved log-scale plot: data/processed_clean_bp_p/jpsik_fit_2024_B2CC_B8_F10215_log.png
Completed processing 2024_B2CC_B8_F10215.root

[#1] INFO:DataHandling -- RooAbsReal::attachToTree(Bp_DTF_OwnPV_MASS) TTree Float_t branch Bp_DTF_OwnPV_MASS will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #1 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5786.48
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #3 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5032.99
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data) Skipping event #4 because Bp_DTF_OwnPV_MASS cannot accommodate the value 5147.56
[#1] INFO:DataHandling

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =      -194507.0167 Edm =        5743.27613 NCalls =     25
Warning in <Minuit2>: MnPosDef Matrix forced pos-def by adding to diagonal 2.37639
Info in <Minuit2>: MnSeedGenerator run Hesse - Initial seeding state: 
  Minimum value : -194507.0167
  Edm           : 18802.74294
  Internal parameters:	[     0.2891274493                0     0.1761089065    -0.1433475689     0.1433475689      1.505265445]	
  Internal gradient  :	[      -414.898093     -197.4628655     -362.0650859     -1178.939289      3536.816256     -4806.554524]	
  Internal covariance matrix:
[[      1.0625517      1.9300779   -0.011529159   -0.043486183     0.14036505  0.00012300218]
 [      1.9300779      3.5154671   -0.020992807   -0.079086862     0.25527715  0.00022050291]
 [   -0.011529159   -0.020992807  0.00019364511  0.00047202331  -0.0015236039 -1.6141675e-06]
 [   -0.04348618